In [1]:
import requests
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



os.environ["OPENAI_API_KEY"]= ""
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_API_KEY= ""


URL="https://leagueoflegends.fandom.com/api.php"

PAGES=[
    "Basic item"
]

def get_lol_pages(limit=500):

    itemsPages=[]
    params = {
        "action": "query",
        "list": "categorymembers",
        "cmtitle": "Category:Items",
        "cmlimit": limit,
        "format": "json"
    }

    while True:
        response = requests.get(URL, params=params)
        data = response.json()

        itemsPages.extend(data["query"]["categorymembers"])

        if "continue" in data:
            params.update(data["continue"])
        else:
            break
    
    return itemsPages

def extract_pages(title):

    params = {
        "action": "parse",
        "page": title,
        "format": "json",
        "prop": "text",
    }

    headers = {"User-Agent": "RAG_project/0.0.1"}

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    if "parse" not in data or "text" not in data["parse"]:
        return None

    return data["parse"]["text"]["*"]

In [2]:
itemsPages=get_lol_pages()

In [3]:
for i in range(len(itemsPages)):
    print(itemsPages[i]["title"])

Active ability items
Basic item
Consumable item
Doran
Epic item
Item (League of Legends)
Legendary item
List of items
Mythic item
Removed items
Starter item
Trinket item
Category:Game modes items
Category:Item icons
Category:Item screenshots
Category:Item SFX
Category:Items by class
Category:Items by effect
Category:Items by release
Category:Items by stat
Category:Items by tier
Category:Map specific balancing
Category:Removed items
Category:Unreleased items


In [4]:
item=[extract_pages(title) for title in PAGES]

In [5]:
print(item[0])

<div class="mw-content-ltr mw-parser-output" lang="en" dir="ltr"><p><b>Basic item</b> is one of the item classifications in <a href="/wiki/League_of_Legends" title="League of Legends">League of Legends</a>. Items of this type are components of <a href="/wiki/Epic_item" title="Epic item">Epic items</a> and <a href="/wiki/Legendary_item" title="Legendary item">Legendary items</a>. They are distinct from <a href="/wiki/Starter_item" title="Starter item">Starter items</a>, which are not the components of any other item.
</p>
<div class="columntemplate" style="-moz-column-count:2; -webkit-column-count:2; column-count:2;">
<ul><li><span class="inline-image label-after item-icon" style="display:inline;white-space:pre;" data-param="" data-item="Amplifying Tome" data-game="lol"><span class="border" style="display:inline-block;position:relative;" data-width="20"><span class="mw-image-border" typeof="mw:File"><a href="/wiki/Amplifying_Tome" title="Amplifying Tome"><img alt="Amplifying Tome" src="

In [6]:
item_pages=[]
for i in range(len(item)):
    dataitem=item[i].split("Champion exclusive items")[0].split('<a href="')[5:]
    for u in range(0,len(dataitem),2):
        item_pages.append(dataitem[u].split('"')[0])
print(len(item_pages))
for i in range(len(item_pages)):
    print(item_pages[i])

122
/wiki/Amplifying_Tome
/wiki/B._F._Sword
/wiki/Blasting_Wand
/wiki/Cloak_of_Agility
/wiki/Cloth_Armor
/wiki/Dagger
/wiki/Faerie_Charm
/wiki/Glowing_Mote
/wiki/Long_Sword
/wiki/Needlessly_Large_Rod
/wiki/Null-Magic_Mantle
/wiki/Pickaxe
/wiki/Rejuvenation_Bead
/wiki/Ruby_Crystal
/wiki/Sapphire_Crystal
/wiki/Item
/wiki/Dark_Seal
/wiki/Doran%27s_Ring
/wiki/Guardian%27s_Blade
/wiki/Guardian%27s_Horn
/wiki/Gustwalker_Hatchling
/wiki/Scorchclaw_Pup
/wiki/World_Atlas
/wiki/Control_Ward
/wiki/Elixir_of_Sorcery
/wiki/Health_Potion
/wiki/Farsight_Alteration
/wiki/Stealth_Ward
/wiki/Elixir_of_Force
/wiki/Eye_of_the_Herald
/wiki/Slightly_Magical_Boots
/wiki/Your_Cut
/wiki/Boots
/wiki/Ionian_Boots_of_Lucidity
/wiki/Plated_Steelcaps
/wiki/Symbiotic_Soles
/wiki/Zephyr
/wiki/B._F._Sword
/wiki/Cloak_of_Agility
/wiki/Dagger
/wiki/Glowing_Mote
/wiki/Needlessly_Large_Rod
/wiki/Pickaxe
/wiki/Ruby_Crystal
/wiki/Aegis_of_the_Legion
/wiki/Bami%27s_Cinder
/wiki/Blighting_Jewel
/wiki/Catalyst_of_Aeons
/wiki/C

In [7]:
from bs4 import BeautifulSoup
import requests

def get_item_stats(url_path):
    title = url_path.split("/wiki/")[1]
    if "%27" in title:
        title=title.split("%27")[0]+"'"+title.split("%27")[1]
    res = requests.get(
        "https://leagueoflegends.fandom.com/api.php",
        params={"action": "parse", "page": title, "format": "json", "prop": "text"},
        headers={"User-Agent": "RAG_project/0.0.1"}
    ).json()

    html = res["parse"]["text"]["*"]
    soup = BeautifulSoup(html, "html.parser").get_text()

    req=soup.split("Requirement")
    if len(req)!=1:
        print("Not a real item")
        return None

    stats=soup.split("Stats")

    if len(stats)!=1:
        stats=stats[1].strip().split("\n\n\n\n\n")[0].strip()
        stats=stats.split("\n\n\n")
    else:
        stats=None
    
    passive=soup.split("Passive")

    if len(passive)!=1:
        passive=passive[1].strip().split("\n\n\n")[0]
    else:
        passive=None

    recipe=soup.split("Recipe")
    if len(recipe)!=1:
        recipe=soup.split("Recipe")[1].strip()
        numbers=recipe.split("Cost\nSell\nID\n\n\n\n\n")[1].strip().split("\n")
        cost=numbers[0].strip()
        sell=numbers[1].strip()
    else:
        recipe=None

    used_in=soup.split("Builds into")
    modes=soup.split("Availability")[1]

    if len(used_in)!=1:
        modes=modes.split("Builds into")[0].strip().split("\n")
        modes=[mode.strip() for mode in modes if mode!=""]
        used_in=used_in[1].split("Menu")[0].strip()
    else:
        modes=modes.split("Menu")[0].strip().split("\n")
        used_in=None

    tags=soup.split("Menu")[1].split("\n\n\n")[0].strip()

    description=soup.split("Menu")[1]

    if len(description.split("Keywords"))!=1:
        description=description.split("Keywords")[1].split("\n\n\n\n")[1]
    else:
        description=description.split("\n\n\n\n")[2]
    description=description.split(".")[0]+"."

    notes=soup.split("Notes")
    if len(notes)!=1:
        notes=notes[1].split("Patch History")[0].strip()

    return title.strip(),stats,passive,cost,sell,modes,used_in,tags,description,notes

print(get_item_stats(item_pages[17]))

("Doran's_Ring", ['+18 ability power', '+90 health'], 'Unique – Restoration: Restore Champions hit by Piltover Peacemakers (Q) at greater than 75% of max range1.25 mana every second. If you cannot gain mana,  heal Champions killed with Comeuppance (R)0.55 health instead.Basic attacks deal Champions killed from outside standard Attack range5 bonus physical damage on-hit against minions.', '400', '160', ['SR 5v5', ' Nexus Blitz'], None, 'MageOn-Hit EffectsAbility PowerMana & RegenerationHealth & Regeneration', "Doran's Ring is a starter item in  League of Legends.", "Restoration's heal is granted instead of its mana either if the user is manaless or at full mana.\nOld icons\n1st version2nd version\nTrivia\n Doran's Ring's Howling Abyss counterpart is  Guardian's Orb.\n Doran's Ring's Dominion counterpart was  Prospector's Ring.")


In [8]:
w=0
dict_item={}
for i in range(len(item_pages)):
    try:
        title,stats,passive,cost,sell,modes,used_in,tags,description,notes=get_item_stats(item_pages[i])
        dict_item[title]={
            "stats":stats,
            "passive":passive,
            "cost":cost,
            "sell":sell,
            "modes":modes,
            "used_in":used_in,
            "tags":tags,
            "description":description,
            "notes":notes
        }
    except:
        w+=1
        print("error,",item_pages[i])

print(w," errors")

error, /wiki/Item
Not a real item
error, /wiki/Gustwalker_Hatchling
Not a real item
error, /wiki/Scorchclaw_Pup
error, /wiki/Control_Ward
Not a real item
error, /wiki/Elixir_of_Sorcery
Not a real item
error, /wiki/Farsight_Alteration
error, /wiki/Stealth_Ward
Not a real item
error, /wiki/Elixir_of_Force
error, /wiki/Eye_of_the_Herald
error, /wiki/Your_Cut
Not a real item
error, /wiki/Zephyr
Not a real item
error, /wiki/Watchful_Wardstone
Not a real item
error, /wiki/Bloodsong
Not a real item
error, /wiki/Bounty_of_Worlds
Not a real item
error, /wiki/Dream_Maker
Not a real item
error, /wiki/Solstice_Sleigh
Not a real item
error, /wiki/Vigilant_Wardstone
error, /wiki/Zaz%27Zak%27s_Realmspike
18  errors
